# DenseNet 구현 및 학습

In [27]:
import tensorflow as tf
import numpy as np

# 초매개변수

In [28]:
EPOCHS = 10

# DenseUnit 구현

In [29]:
class DenseUnit(tf.keras.Model):
    def __init__(self, filter_out, kernel_size):
        super(DenseUnit,self).__init__()
        self.bn = tf.keras.layers.BatchNormalization()
        self.conv = tf.keras.layers.Conv2D(filter_out,kernel_size,padding='same')
        self.concat = tf.keras.layers.Concatenate()
        
    def call(self,x,training=False,mask=None): # x : (Batch,H,W,C)
        h = self.bn(x,training=training)
        h = tf.nn.relu(h)
        h = self.conv(h) # (Batch, H, W, filter_out)
        return self.concat([x,h]) # (Batch, H, W, (C + filter_out))

# DenseLayer 구현

In [30]:
class DenseLayer(tf.keras.Model):
    
    def __init__(self, num_unit, growth_rate, kernel_size):
        super(DenseLayer,self).__init__()
        self.sequence = list()
        for idx in range(num_unit):
            self.sequence.append(DenseUnit(growth_rate,kernel_size))
            
    def call(self, x,training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training=training)
        return x

# Transition Layer 구현

In [31]:
class TransitionLayer(tf.keras.Model):
    def __init__(self,filters,kernel_size):
        super(TransitionLayer,self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters,kernel_size,padding='same')
        self.pool = tf.keras.layers.MaxPool2D()
        
    def call(self,x,training=False,mask=None):
        x = self.conv(x)
        return self.pool(x)

# 모형 정의

In [32]:
class DenseNet(tf.keras.Model):
    def __init__(self):
        super(DenseNet,self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(8,(3,3),padding='same',activation='relu')
        self.dl1 = DenseLayer(2,4,(3,3)) # 28 x 28 x 16
        self.tr1 = TransitionLayer(16,(3,3)) # 14 x 14 x 16
    
        self.dl2 = DenseLayer(2,8,(3,3)) # 14 x 14 x 32
        self.tr2 = TransitionLayer(32,(3,3)) # 7 x 7 x 32
        
        self.dl3 = DenseLayer(2,16,(3,3)) # 7 x 7 x 64
        
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10,activation='softmax')
        
    def call(self,x,training=False,mask=None):
        x = self.conv1(x)
        x = self.dl1(x,training=training)
        x = self.tr1(x)
        x = self.dl2(x,training=training)
        x = self.tr2(x)
        x = self.dl3(x,training=training)
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)
        
        
        

# Train and Test

## Dataset

In [33]:
mnist = tf.keras.datasets.mnist

(X_train,y_train),(X_test,y_test) = mnist.load_data()

X_train, X_test = X_train / 255., X_test / 255.

X_train = X_train[...,tf.newaxis].astype(np.float32)
X_test = X_test[...,tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(32)

## 학습 환경 정의

### 모형 생성, 손실 함수, 최적화 ,평가지표

In [34]:
model = DenseNet() 

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [35]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
  with tf.GradientTape() as tape:
    predictions = model(images,training=True) 
    loss = loss_object(labels, predictions)

  gradients = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels,predictions)

@tf.function
def test_step(model,images,labels,loss_object,test_loss,test_accuracy): 
  predictions = model(images,training=False)
  
  t_loss = loss_object(labels,predictions)
  test_loss(t_loss)
  test_accuracy(labels,predictions)


### 학습

In [36]:
for epoch in range(EPOCHS): 
  for images, labels in train_ds:
    train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

  for test_images, test_labels in test_ds: 
    test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

  template = 'Epoch {}, Loss : {}, Accuracy : {}, Test Loss : {}, Test Accuracy : {}'
  print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100, test_loss.result(),
                        test_accuracy.result() * 100))

  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()


Epoch 1, Loss : 0.1154453381896019, Accuracy : 96.65333557128906, Test Loss : 0.06464579701423645, Test Accuracy : 98.20999908447266
Epoch 2, Loss : 0.0587044395506382, Accuracy : 98.36000061035156, Test Loss : 0.055165208876132965, Test Accuracy : 98.30999755859375
Epoch 3, Loss : 0.04389728233218193, Accuracy : 98.73666381835938, Test Loss : 0.09075435250997543, Test Accuracy : 97.98999786376953
Epoch 4, Loss : 0.04317108169198036, Accuracy : 98.8699951171875, Test Loss : 0.05866210535168648, Test Accuracy : 98.56999969482422
Epoch 5, Loss : 0.038482531905174255, Accuracy : 98.98999786376953, Test Loss : 0.06209498271346092, Test Accuracy : 98.41999816894531
Epoch 6, Loss : 0.031786274164915085, Accuracy : 99.11000061035156, Test Loss : 0.0585537813603878, Test Accuracy : 98.63999938964844
Epoch 7, Loss : 0.03048602305352688, Accuracy : 99.19833374023438, Test Loss : 0.04903595149517059, Test Accuracy : 98.86000061035156
Epoch 8, Loss : 0.026415957137942314, Accuracy : 99.32167053222